# Import Train and Test datasets

In [1]:
from deepchem.data import DiskDataset

train = DiskDataset("aqsoldb_train")
test = DiskDataset("aqsoldb_test")


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


<DiskDataset X.shape: (7870,), y.shape: (7870,), w.shape: (7870,), task_names: [0]>


# Create train and test datasets

In [2]:
from torch_geometric.data import Dataset
from deepchem.data.pytorch_datasets import _TorchDiskDataset


class AqSolDBDataset(Dataset):
  
  def __init__(self, deepchem_dataset: _TorchDiskDataset):
    self.graph_list = [mol.to_pyg_graph() for mol, _, _, _ in deepchem_dataset]
    self.labels = [y for _, y, _, _ in deepchem_dataset]
    self.length = len(self.labels)
    self._indices = None

  def __getitem__(self, i):
    graph = self.graph_list[i]
    label = self.labels[i]
    return graph, label
  
  def __len__(self):
    return self.length
  
  def len(self):
    return len(self)
  
  def get(self, idx):
    graph = self.graph_list[idx]
    label = self.labels[idx]
    return graph, label

train_dataset = AqSolDBDataset(train.make_pytorch_dataset())
test_dataset = AqSolDBDataset(test.make_pytorch_dataset())

train_dataset

AqSolDBDataset(7870)

# Create AqSol model

In [3]:
import torch.nn as nn
import torch_geometric.nn as pyg_nn
import torch
from torch_geometric.nn import global_add_pool
import torch.nn.functional as F


class AqSolModel(nn.Module):
  def __init__(
      self,
      n_features,
      hidden_channels,
      lr=10**-3,
      weight_decay=10**-2.5,
      dropout=0.2
    ):
    super(AqSolModel, self).__init__()

    self.conv1 = pyg_nn.GCNConv(n_features, hidden_channels)
    self.conv2 = pyg_nn.GCNConv(hidden_channels, hidden_channels)
    self.conv3 = pyg_nn.GCNConv(hidden_channels, int(hidden_channels / 2))
    
    self.lin = nn.Linear(int(hidden_channels / 2), int(hidden_channels / 2))
    self.lin2 = nn.Linear(int(hidden_channels / 2), 1)
    
    self.loss = nn.MSELoss()
    self.optimizer = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)
    self.dropout = dropout

  def forward(self, mol):
    mol_x, mol_edge_index = mol.x, mol.edge_index
    
    mol_x = self.conv1(mol_x, mol_edge_index)
    mol_x = mol_x.relu()
    mol_x = self.conv2(mol_x, mol_edge_index)
    mol_x = mol_x.relu()
    mol_x = self.conv3(mol_x, mol_edge_index)
    mol_x = mol_x.relu()

    mol_x = global_add_pool(mol_x, mol.batch)

    mol_x = F.dropout(mol_x, p=self.dropout, training=self.training)

    mol_x = self.lin(mol_x)
    mol_x.relu()
    mol_x = self.lin2(mol_x)
    return mol_x

# Create model and train

In [ ]:
from torch_geometric.loader import DataLoader
import math
import numpy as np
from tqdm import tqdm


model = AqSolModel(30, 128)

batch_size = min(len(train_dataset), 64)
num_epochs = 50
num_batches = math.ceil(len(train_dataset) / batch_size)
losses = np.zeros(num_epochs)
mean_loss = 0

model.train()

epoch_iter = range(num_epochs)

for epoch in epoch_iter:
  # print("Epoch " + str(epoch + 1) + " of " + str(num_epochs))
  model.optimizer.zero_grad()
  epoch_loss = 0
  for i, batch in enumerate(DataLoader(train_dataset, batch_size=batch_size)):
    graphs, labels = batch
    pred = model(graphs)
    actual = labels.reshape((len(labels), 1))
    # print(pred, actual)
    loss = model.loss(pred, actual)
    # print(loss.item())
    epoch_loss += loss.item()
    loss.backward()
    model.optimizer.step()
  losses[epoch] = epoch_loss / num_batches
  mean_loss += losses[epoch]
  print("loss: " + str(epoch_loss))


# Validate with test set

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

model.eval()
preds = np.zeros((len(test)))
for i, (x_test, y_test) in enumerate(zip(test.X, test.y)):
    pred = model(x_test.to_pyg_graph()).detach().numpy()
    preds[i] = pred

print(preds, test.y)
print(mean_squared_error(test.y, preds))
print(mean_absolute_error(test.y, preds))

from matplotlib import pyplot as pyt

pyt.hist([preds, test.y], label=["Prediction", "Actual"]);
pyt.legend()

# Validate against train

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

model.eval()
preds = np.zeros((len(train)))
for i, (x_test, y_test) in enumerate(zip(train.X, train.y)):
    pred = model(x_test.to_pyg_graph()).detach().numpy()
    preds[i] = pred

print(preds, train.y)
print(mean_squared_error(train.y, preds))
print(mean_absolute_error(train.y, preds))

In [ ]:
from matplotlib import pyplot as pyt

pyt.hist([preds, train.y], label=["Prediction", "Actual"]);
pyt.legend()

# Hyperparameter Sweep

In [6]:
import wandb
from torch_geometric.loader import DataLoader


class Trainer:
  
  def __init__(self, model, dataset, batch_size):
    self.model = model
    self.dataset = dataset
    self.batch_size = batch_size
    
    self.mean_loss = 0
    self.run_epochs = 0
    
  def train_one_epoch(self):
    self.model.optimizer.zero_grad()
    epoch_loss = 0
    for i, batch in enumerate(DataLoader(self.dataset, batch_size=self.batch_size)):
      graphs, labels = batch
      pred = self.model(graphs)
      actual = labels.reshape((len(labels), 1))
      loss = self.model.loss(pred, actual)
      epoch_loss += loss.item()
      loss.backward()
      self.model.optimizer.step()
    self.mean_loss += epoch_loss
    print("loss: " + str(epoch_loss))
    return epoch_loss
    
  def run(self, num_epochs):
    epoch_loss = 0
    for _ in range(num_epochs):
      epoch_loss = self.train_one_epoch()
    wandb.log({"mse": epoch_loss})


In [ ]:
from torch_geometric.loader import DataLoader
import math
import numpy as np
from tqdm import tqdm


model = AqSolModel(30, 128)

batch_size = min(len(train_dataset), 64)
num_epochs = 10
num_batches = math.ceil(len(train_dataset) / batch_size)
losses = np.zeros(num_epochs)
mean_loss = 0

wandb.init(
    # set the wandb project where this run will be logged
    project="SolubilityPredictor",
    
    # track hyperparameters and run metadata
    config={
      "learning_rate": 10**-2,
      "architecture": "ConvGNN",
      "epochs": num_epochs,
    }
)

model.train()

epoch_iter = range(num_epochs)

for epoch in epoch_iter:
  # print("Epoch " + str(epoch + 1) + " of " + str(num_epochs))
  model.optimizer.zero_grad()
  epoch_loss = 0
  for i, batch in enumerate(DataLoader(train_dataset, batch_size=batch_size)):
    graphs, labels = batch
    pred = model(graphs)
    actual = labels.reshape((len(labels), 1))
    # print(pred, actual)
    loss = model.loss(pred, actual)
    # print(loss.item())
    epoch_loss += loss.item()
    loss.backward()
    model.optimizer.step()
  losses[epoch] = epoch_loss / num_batches
  mean_loss += losses[epoch]
  print("loss: " + str(epoch_loss))
  wandb.log({"loss": epoch_loss})


wandb.finish()

In [8]:
from torch_geometric.loader import DataLoader


sweep_config = {
  "name": "sweep",
  "method": "bayes",
  "metric": {
    "goal": "minimize",
    "name": "mse"
  },
  "parameters": {
    "hidden_channels": {
      "min": 30,
      "max": 512
    },
    "num_epochs": {
      "min": 1,
      "max": 5
    },
    "batch_size": {
      "min": 1,
      "max": 128
    },
    "lr": {
      "min": 1e-6,
      "max": 1e-1
    },
    "weight_decay": {
      "min": float(0),
      "max": 1e-5
    },
    "dropout": {
      "min": 0,
      "max": 1
    }
  }
}


sweep_id = wandb.sweep(
  sweep_config, project="SolubilityPredictor"
)

def tune_hyperparameters(config=None):
  with wandb.init(config=config):
    config = wandb.config
    model = AqSolModel(
      30,
      config.hidden_channels,
      lr=config.lr,
      weight_decay=config.weight_decay,
      dropout=config.dropout
    )
    trainer = Trainer(model, train_dataset, config.batch_size)
    trainer.run(config.num_epochs)

wandb_config = {
  "architecture": "ConvGNN",
}


wandb.agent(
  sweep_id,
  function=tune_hyperparameters,
  project="SolubilityPredictor",
  count=10
)

Create sweep with ID: 5d4kipr4
Sweep URL: https://wandb.ai/alexb02/SolubilityPredictor/sweeps/5d4kipr4
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: oxo9th0s with config:
wandb: 	batch_size: 113
wandb: 	hidden_channels: 116
wandb: 	lr: 0.0031676241864073535
wandb: 	num_epochs: 6
wandb: 	weight_decay: 1.6646060646142023e-06


loss: 27.930533476173878
loss: 11.686980627477169
loss: 6.548539321869612
loss: 3.1140586379915476
loss: 10.998607411980629
loss: 9.385794211179018


mse,▁
mse,9.38579


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 7fhkqvjp with config:
wandb: 	batch_size: 95
wandb: 	hidden_channels: 211
wandb: 	lr: 0.040101871318269235
wandb: 	num_epochs: 2
wandb: 	weight_decay: 3.392825869283121e-06


...

loss: 27.765543213114142
loss: 486.75018855929375
loss: 344.2917249780148
loss: 255.42646943219006
loss: 162.3536927383393
loss: 20.022428223863244
loss: 76.62445028312504
loss: 36.92621922492981
loss: 34.292299365624785


mse,▁
mse,34.2923


wandb: Agent Starting Run: 3cjdfpxd with config:
wandb: 	batch_size: 19
wandb: 	hidden_channels: 443
wandb: 	lr: 0.00108399110375668
wandb: 	num_epochs: 10
wandb: 	weight_decay: 1.5145354410457838e-07


loss: 591.1104813748971
loss: 60.31645524781197
loss: 194.32123770192266
loss: 43.25601611658931
loss: 46.76056456193328
loss: 43.130606949329376
loss: 50.614115483127534
loss: 106.65383637323976
loss: 457.7980485474691
loss: 1234.213883771561


mse,▁
mse,1234.21388
